In [67]:
from unidecode import unidecode
import pandas as pd
import os

In [68]:
dataframes = []

for archivo in os.listdir("./data/output/places"):
    if archivo.endswith(".csv"):
        df = pd.read_csv(os.path.join("./data/output/places", archivo))
        dataframes.append(df)

df_combined = pd.concat(dataframes, ignore_index=True).drop("Unnamed: 0", axis = 1)
df_combined.head()

,name,desc,score,c_score,price,category,accessibility,schedule,web,search_parameters,phone,address,lat,lon
0,Los Fabio's Popular,Información Opciones de servicio\n\nAsientos...,4.5,74.0,$ 10.000-20.000,Hamburguesería,NaN,"6 pm.,9 pm.",NaN,Comuna Popular Restaurantes,319 6117987,"Cra. 42c #107-001, La Isla, Medellín, Popular,...",6.295462,-75.548500
1,BRASAS MI SOR,Información Opciones de servicio\n\nRetiros ...,5.0,1.0,NaN,Restaurante,NaN,NaN,https://brasas-mi-sor.ola.click/,Comuna Popular Restaurantes,314 7757452,"Cra. 43 #110 a 58, La Isla, Medellín, Popular,...",6.299469,-75.547267
2,Mandingas la 107,Información Opciones de servicio\n\nPara lle...,4.8,20.0,$ 1-10.000,Restaurante de comida para llevar,NaN,"6 pm.,6 .",NaN,Comuna Popular Restaurantes,319 3353560,"Cra 49B #107-3, Villa Niza, Medellín, Santa Cr...",6.297972,-75.554448
3,Grosseto Pizzeria,Información Accesibilidad\n\nEntrada accesib...,4.4,14.0,NaN,Restaurante,Accesible con silla de ruedas,"12 pm.,6 .",NaN,Comuna Popular Restaurantes,324 2886618,"Calle 126, Cra. 42ee #88, Medellín, Antioquia",6.304309,-75.546252
4,Las Paisanas,Información Opciones de servicio\n\nAsientos...,4.6,15.0,NaN,Restaurante,NaN,NaN,NaN,Comuna Popular Restaurantes,302 3874084,"0505, Medellín, Antioquia",6.305670,-75.553140


In [69]:
df_filtered = df_combined.copy()
df_filtered = df_filtered[df_filtered["score"] > 3]
df_filtered = df_filtered[df_filtered["c_score"] > 10]
df_filtered = df_filtered[df_filtered["name"].apply(lambda x: "infantil" not in unidecode(x).lower())]
df_filtered = df_filtered[df_filtered["search_parameters"].apply(lambda x: x.split()[2:][0] != "Hoteles")]
len(df_filtered)

1345

In [70]:
df_filtered["bayesian_mean"] = (df_filtered["score"] * df_filtered["c_score"] + 4 * 10) / (df_filtered["c_score"] + 10)

In [71]:
df_filtered["Comuna"] = df_filtered["search_parameters"].apply(lambda x: x.split()[1])

In [72]:
df_filtered.head()

,name,desc,score,c_score,price,category,accessibility,schedule,web,search_parameters,phone,address,lat,lon,bayesian_mean,Comuna
0,Los Fabio's Popular,Información Opciones de servicio\n\nAsientos...,4.5,74.0,$ 10.000-20.000,Hamburguesería,NaN,"6 pm.,9 pm.",NaN,Comuna Popular Restaurantes,319 6117987,"Cra. 42c #107-001, La Isla, Medellín, Popular,...",6.295462,-75.548500,4.440476,Popular
2,Mandingas la 107,Información Opciones de servicio\n\nPara lle...,4.8,20.0,$ 1-10.000,Restaurante de comida para llevar,NaN,"6 pm.,6 .",NaN,Comuna Popular Restaurantes,319 3353560,"Cra 49B #107-3, Villa Niza, Medellín, Santa Cr...",6.297972,-75.554448,4.533333,Popular
3,Grosseto Pizzeria,Información Accesibilidad\n\nEntrada accesib...,4.4,14.0,NaN,Restaurante,Accesible con silla de ruedas,"12 pm.,6 .",NaN,Comuna Popular Restaurantes,324 2886618,"Calle 126, Cra. 42ee #88, Medellín, Antioquia",6.304309,-75.546252,4.233333,Popular
4,Las Paisanas,Información Opciones de servicio\n\nAsientos...,4.6,15.0,NaN,Restaurante,NaN,NaN,NaN,Comuna Popular Restaurantes,302 3874084,"0505, Medellín, Antioquia",6.305670,-75.553140,4.360000,Popular
5,Ricuras Jadi,Información Accesibilidad\n\nEntrada accesib...,4.4,445.0,$ 20.000-30.000,Restaurante,Accesible con silla de ruedas,"12 pm.,6 .",NaN,Comuna Popular Restaurantes,301 1856998,"Cl. 116a, Las Brisas, Medellín, Castilla, Mede...",6.310462,-75.563710,4.391209,Popular


In [73]:
print("Datos iniciales:", len(df_combined))
print("Datos filtrados:", len(df_filtered))

Datos iniciales: 1781
Datos filtrados: 1345


In [74]:
df_filtered.to_csv("./data/output/places/final_places.csv")